In [29]:
import numpy as np

import cv2 as cv2

import tensorflow as tf

tf.compat.v1.disable_eager_execution()
from tensorflow import keras
import os
import random

In [4]:
# 预先写几个方法
# 方法一就是获取文件夹下的数据
def get_imgs(rate = 0.2):
    """
    获取图片，并划分训练集和测试集
    Parameters:
        rate:测试集 2 和训练集 10 的比例，即测试集个数/训练集个数
    Returns:
        test_imgs:测试集
        test_labels:测试集标签
        train_imgs:训练集
        train_labels:训练集标签
    """
    # 读取图片
    imgs = os.listdir('./verify/')
    # 打乱图片顺序
    random.shuffle(imgs)

    # 数据集总共个数
    imgs_num = len(imgs)
    
    # 按照比例求出测试集个数
    test_num = int(imgs_num * rate / (1 + rate))
    
    # 测试集，测试数据的路径
    test_imgs = imgs[:test_num]
    # 根据文件名获取测试集标签
    test_labels = list(map(lambda x: x.split('.')[0], test_imgs))
    
    
    # 训练集
    train_imgs = imgs[test_num:]
    # 根据文件名获取训练集标签
    train_labels = list(map(lambda x: x.split('.')[0], train_imgs))

    return test_imgs, test_labels, train_imgs, train_labels

In [5]:

def text2vec(text):
    char_set_len = 63
    """
    文本转向量
    Parameters:
        text:文本
    Returns:
        vector:向量
    """
    if len(text) > 4:
        raise ValueError('验证码最长4个字符')

    vector = np.zeros(4 * char_set_len)
    def char2pos(c):
        if c =='_':
            k = 62
            return k
        k = ord(c) - 48
        if k > 9:
            k = ord(c) - 55
            if k > 35:
                k = ord(c) - 61
                if k > 61:
                    raise ValueError('No Map')
        return k
    for i, c in enumerate(text):
        idx = i * char_set_len + char2pos(c)
        vector[idx] = 1
    return vector

In [6]:
# 定义了每次取出一批数据，tensorflow训练的时候，一批批喂给算法，for循环执行很多次
def get_next_batch(test_imgs,test_labels,train_imgs,train_labels,train_flag=True, batch_size=100):
    train_size = 4160
    test_size = 831
    train_ptr = 0
    test_ptr = 0
    height = 30
    width = 100
    max_captcha = 4
    # 0~ 9（10），a~z（26） ，A~Z（26） --------> 62 + 1 (_未知) -----> 63
    char_set_len = 63
    data_path = './verify/'
    """
    获得batch_size大小的数据集
    Parameters:
        batch_size:batch_size大小
        train_flag:是否从训练集获取数据
    Returns:
        batch_x:大小为batch_size的数据x
        batch_y:大小为batch_size的数据y
        image(用于测试)
    """
    # 从训练集获取数据
    if train_flag == True:
        if (batch_size + train_ptr) < train_size:
            trains = train_imgs[train_ptr:(train_ptr + batch_size)]
            labels = train_labels[train_ptr:(train_ptr + batch_size)]
            train_ptr += batch_size
        else:
            new_ptr = (train_ptr + batch_size) % train_size
            trains = train_imgs[train_ptr:] + train_imgs[:new_ptr]
            labels = train_labels[train_ptr:] + train_labels[:new_ptr]
            train_ptr = new_ptr

#       返回数据，给了形状
        batch_X = np.zeros([batch_size, height*width])
#       目标值，独热编码表示 4 * 63 ------> 概率问题
        batch_y = np.zeros([batch_size, max_captcha*char_set_len])

        for index, train in enumerate(trains):
            # 黑白图片
            img = np.mean(cv2.imread(data_path + train), axis = -1)
            # 将多维降维1维
            batch_X[index,:] = img.flatten() / 255
        for index, label in enumerate(labels):
            batch_y[index,:] = text2vec(label)

    # 从测试集获取数据
    else:
        if (batch_size + test_ptr) < test_size:
            tests = test_imgs[test_ptr:(test_ptr + batch_size)]
            labels = test_labels[test_ptr:(test_ptr + batch_size)]
            test_ptr += batch_size
        else:
            new_ptr = (test_ptr + batch_size) % test_size
            tests = test_imgs[test_ptr:] + test_imgs[:new_ptr]
            labels = test_labels[test_ptr:] + test_labels[:new_ptr]
            test_ptr = new_ptr

        batch_X = np.zeros([batch_size, height*width])
        batch_y = np.zeros([batch_size, max_captcha*char_set_len])

        for index, test in enumerate(tests):
#             图片灰度化处理，黑白处理
            img = np.mean(cv2.imread(data_path + test), axis = -1)
            # 将多维降维1维
            batch_X[index,:] = img.ravel() / 255
        for index, label in enumerate(labels):
            batch_y[index,:] = text2vec(label)
        return batch_X, batch_y
    return batch_X, batch_y

In [7]:
def vec2text(vec):
    char_set_len = 63
    """
    向量转文本
    Parameters:
        vec:向量
    Returns:
        文本
    """
    char_pos = vec.nonzero()[0]
    text = []
    for c in char_pos:
        char_idx = c % char_set_len
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx < 36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)

In [8]:
test_imgs, test_labels, train_imgs, train_labels=get_imgs()

In [17]:
np.array(test_labels)

array(['67go', 'p7fk', 'q4n8', 'st1l', 'y7zd', 'xgkk', 'ku1n', 'k9s2',
       'bfug', '4gxe', '8lxy', 'u79s', 'tope', 'ntc1', '5do7', 'qths',
       'lpzn', '5dsi', '466c', '6nt0', 'xef1', 'nupp', '793d', '5dj3',
       '2ohi', 'ijgp', 'o0pu', 'avf0', 'ev0w', 'ik57', 'in1n', 'pcon',
       'lobs', 'alt1', 'cequ', '4im4', 'zpv1', 'iemp', '8w7d', 'pt1a',
       '84a3', 'dx7n', 'q3d9', '2vf6', '26jc', 'a9oe', '81pi', 'hdvh',
       'pzwi', 'w0lj', 'mr6a', 'a86p', 'p7u6', 'm6i9', 'tkxk', '9p3o',
       '9xrc', '4adf', 'c9rw', 'nard', '60zs', 'sk6y', '67yi', 'ahmx',
       '3wy6', 'dja7', 'mvy1', '1m7l', '5iwd', '6hb3', '6e30', '0f5z',
       '1kmd', 'cqoc', '5ayl', 'zm3e', 'su0u', 'zyr8', 'spem', 'jo7m',
       '3234', '6woe', 'j0gg', 'yn26', 'trzl', 'dain', 'wsyc', '067z',
       'ar79', 'ue3c', 'fx3q', 'y8ps', '5et6', '5yju', '96vl', 'hbof',
       'cts9', 'q3n3', '0d7n', '381l', 'yrfc', 'l1g2', 'kbj0', '20dl',
       'at6k', 'yocl', 'afpg', 'ghf2', 'go1b', '0hun', 'ruyp', '714d',
      

In [35]:
height = 30
width = 100
max_captcha = 4
char_set_len = 63
x_test = np.zeros([np.array(test_imgs).shape[0], height*width])
y_test = np.zeros([np.array(test_labels).shape[0], max_captcha*char_set_len])
for index, train in enumerate(test_imgs):
            # 黑白图片
            img = np.mean(cv2.imread('./verify/' + train), axis = -1)
            # 将多维降维1维
            x_test[index,:] = img.flatten() / 255.0
for index, label in enumerate(test_labels):
            y_test[index,:] = text2vec(label)

In [36]:
height = 30
width = 100
max_captcha = 4
char_set_len = 63
x_train = np.zeros([np.array(train_imgs).shape[0], height*width])
y_train = np.zeros([np.array(train_labels).shape[0], max_captcha*char_set_len])
for index, train in enumerate(train_imgs):
            # 黑白图片
            img = np.mean(cv2.imread('./verify/' + train), axis = -1)
            # 将多维降维1维
            x_train[index,:] = img.flatten() / 255.0
for index, label in enumerate(train_labels):
            y_train[index,:] = text2vec(label)

In [20]:
x_test.shape 

(831, 3000)

In [23]:
y_test.shape

(831, 252)

In [50]:
x_test=x_test.reshape(-1,30,100,1)

In [51]:
x_test.shape

(831, 30, 100, 1)

In [52]:
x_train=x_train.reshape(-1,30,100,1)

In [54]:
x_train.shape

(4160, 30, 100, 1)

(30, 100, 1)

In [56]:
model = tf.keras.models.Sequential([
            # 卷积层1 32个 5*5*3的filter，strides=1  padding=same
#             tf.keras.layers.Flatten(input_shape=(30, 100)),
            # [None, 30 * 100]
            tf.keras.layers.Conv2D(
                filters=32, input_shape=x_train[0].shape,kernel_size=3, strides=1, padding="same",
                data_format="channels_last", activation=tf.nn.relu
            ),
            # 池化层1 2*2窗口  strides=2
            tf.keras.layers.MaxPool2D(
                pool_size=2,
                strides=2,
                padding='same',
            ),
            tf.keras.layers.Dropout(0.25),
            # 卷积层2 64个 5*5*32的filter，strides=1  padding=same
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=1, padding="same",
                data_format="channels_last", activation=tf.nn.relu
            ),

            # 池化层2 2*2窗口  strides=2 [none 8 8 64]
            tf.keras.layers.MaxPool2D(
                pool_size=2,
                strides=2,
                padding='same',
            ),
            tf.keras.layers.Dropout(0.25),

            # 全连接层神经网络
            # [none 8 8 64] ----> [None,8*8*64]
            tf.keras.layers.Flatten(),
            # 1024个神经元网络
            tf.keras.layers.Dense(units=1024, activation=tf.nn.relu),
            # 100个神经元网络
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Dense(units=4*63, activation=tf.nn.softmax),

        ])

In [57]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [61]:
x_train.shape

(4160, 30, 100, 1)

In [62]:
tf.compat.v1.disable_eager_execution()


config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)
model.fit(x=x_train, y=y_train,
                       batch_size=32,
                       epochs=1)

Train on 4160 samples


d:\python\venv(data_analysis)\lib\site-packages\tensorflow\python\client\session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Error while reading resource variable conv2d_4/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/conv2d_4/kernel/class tensorflow::Var does not exist.
	 [[{{node conv2d_4/Conv2D/ReadVariableOp}}]]
	 [[dropout_4/cond/then/_30/dropout/random_uniform/RandomUniform/_181]]
  (1) Failed precondition: Error while reading resource variable conv2d_4/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/conv2d_4/kernel/class tensorflow::Var does not exist.
	 [[{{node conv2d_4/Conv2D/ReadVariableOp}}]]
0 successful operations.
0 derived errors ignored.

In [17]:
# X =tf.compat.v1.placeholder(dtype=tf.float64,shape = [None,3000])

# kp = tf.compat.v1.placeholder(dtype=tf.float64,shape = None)

# # 独热编码，长度4（验证码长度）* 63（0~9，A~Z，a~z _ 63）
# y = tf.compat.v1.placeholder(dtype=tf.float64,shape = [None,4*63])
# def crack_captcha_cnn():
    
# #     第一层
#     input_data = tf.reshape(X,shape = [-1,30,100,1])

#     filter1 = tf.Variable(initial_value=tf.random.normal(shape = [3,3,1,32],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
    
#     b1 = tf.Variable(initial_value=tf.random.normal(shape = [32],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
    
#     conv1 = tf.nn.conv2d(input_data,filter1,strides=[1,1,1,1],padding='SAME') + b1
    
#     relu1 = tf.nn.relu(conv1)
# #     池化
#     pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# #     pool1 -------> shape = [-1,15,50,32]
# #     第二层
#     filter2 = tf.Variable(initial_value=tf.random.normal(shape = [3,3,32,64],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
    
#     b2 = tf.Variable(initial_value=tf.random.normal(shape = [64],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)

#     conv2 = tf.nn.conv2d(pool1,filter2,strides=[1,1,1,1],padding='SAME') + b2
    
# #     激活函数
#     sigmoid2 = tf.nn.sigmoid(conv2)
    
# #     池化
#     pool2 = tf.nn.max_pool(sigmoid2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# #     pool2  --------> shape = [-1,8,25,64]
# #     第三层
#     filter3 = tf.Variable(initial_value=tf.random.normal(shape = [3,3,64,64],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
    
#     b3 = tf.Variable(initial_value=tf.random.normal(shape = [64],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
    
#     conv3 = tf.nn.conv2d(pool2,filter3,strides=[1,1,1,1],padding='SAME') + b3

#     relu3 = tf.nn.relu(conv3)
    
#     pool3 = tf.nn.max_pool(relu3,ksize= [1,2,2,1],strides=[1,2,2,1],padding='SAME')
# #     pool3  --------> shape = [-1,4,13,64]
# #     全连接层 1024 个神经元
#     dense = tf.reshape(pool3,shape = [-1,4*13*64])
    
#     conn_w = tf.Variable(tf.random.normal(shape = [4*13*64,1024],dtype = tf.float64,stddev = 0.01),dtype= tf.float64)
    
#     conn_b  = tf.Variable(tf.random.normal(shape = [1024],dtype = tf.float64,stddev = 0.01),dtype= tf.float64)
    
#     conn = tf.matmul(dense,conn_w) + conn_b
# #     conn -------> shape = [-1,1024]
#     conn_relu = tf.nn.relu(conn)
    
# #     dropout
#     dropout = tf.nn.dropout(conn_relu,rate=kp)
    
# #     输出层 y进行对比
#     out_w = tf.Variable(initial_value=tf.random.normal(shape = [1024,4*63],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
#     out_b = tf.Variable(initial_value=tf.random.normal(shape = [4*63],dtype = tf.float64,stddev = 0.01),dtype=tf.float64)
    
#     out = tf.matmul(dropout,out_w) + out_b
    
# #     out ----------> shape = [?,4*63]
#     return out

In [18]:
# # 训练
# def train_crack_captcha_cnn():
    
    
    
#     out = crack_captcha_cnn()
#     saver = tf.compat.v1.train.Saver()
    
# #     真实分布，y 
#     cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels= y,logits=out))
    
#     optimizer = tf.compat.v1.train.AdamOptimizer().minimize(cost)
    
    
# #     计算准确率方法,概率
#     prob_ = tf.nn.softmax(out)
    
#     accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prob_,axis = -1),tf.argmax(y,axis = -1)),tf.float64))
    
    
#     with tf.compat.v1.Session() as sess:
        
#         sess.run(tf.compat.v1.global_variables_initializer())
        
#         test_imgs,test_labels,train_imgs,train_labels = get_imgs()
        
#         for i in range(100000):
            
#             if (i%500 == 0)&(i!=0):
#                 test_imgs,test_labels,train_imgs,train_labels = get_imgs()
            
#             X_train,y_train = get_next_batch(test_imgs,test_labels,train_imgs,train_labels)
            
            
#             optimizer_,cost_ = sess.run(fetches = [optimizer,cost],feed_dict = {X:X_train,y:y_train,kp:0.5})
            
#             if i%100 == 0:
#                 X_test,y_test = get_next_batch(test_imgs,test_labels,train_imgs,train_labels,train_flag=False,batch_size=500)
#                 score = sess.run(fetches = accuracy,feed_dict={X:X_test,y:y_test,kp:1})
#                 print('算法执行次数：{}算法准确率：{}'.format(i,score))
#                 if score >=0.96:
#                     saver.save(sess,'./captcha/model',i)
                    
#             if (i%1000 == 0)&(i!=0):
#                 saver.save(sess,'./captcha/model',i)

In [19]:
# train_crack_captcha_cnn()

NotFoundError: 2 root error(s) found.
  (0) Not found: No algorithm worked!
	 [[node Conv2D_3 (defined at <ipython-input-17-e56fe0ce0afb>:16) ]]
  (1) Not found: No algorithm worked!
	 [[node Conv2D_3 (defined at <ipython-input-17-e56fe0ce0afb>:16) ]]
	 [[Mean_2/_9]]
0 successful operations.
0 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node Conv2D_3:
 Reshape_2 (defined at <ipython-input-17-e56fe0ce0afb>:10)

Input Source operations connected to node Conv2D_3:
 Reshape_2 (defined at <ipython-input-17-e56fe0ce0afb>:10)

Original stack trace for 'Conv2D_3':
  File "D:\Python\Python38\lib\runpy.py", line 193, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\Python\Python38\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\python\venv(data_analysis)\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\python\venv